<a href="https://colab.research.google.com/github/PedroAvilaPaiDaManu/engenharia-dados/blob/main/engenheiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-cloud-storage


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import auth
auth.authenticate_user()


In [7]:
!gcloud init


Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  region: us-east4
  zone: us-east4-a
core:
  account: pedronp.avila88@gmail.com
  project: curso-dataflow-beam-383711

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for this 
configuration:
 [1] pedronp.avila88@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [pedronp.avila88@gm

In [16]:
import os

directory = "engenheiro-dados"
if not os.path.exists(directory):
    os.makedirs(directory)
print(directory)

engenheiro-dados


In [20]:
from google.cloud import storage

# Crie uma instância do cliente de armazenamento do Google Cloud
client = storage.Client()

# Especifique o nome do seu bucket
bucket_name = "engenheiro-dados"

# Recupere uma referência ao bucket
bucket = client.get_bucket(bucket_name)

# Especifique a lista de nomes de arquivo que você deseja baixar do bucket
nomes_arquivos = ["Person.Person.csv", "Production.Product.csv","Sales.Customer.csv","Sales.SalesOrderDetail.csv","Sales.SalesOrderHeader.csv","Sales.SpecialOfferProduct.csv"]

# Itere sobre a lista de nomes de arquivo e baixe cada arquivo para o diretório atual
for nome_do_arquivo in nomes_arquivos:
    # Recupere uma referência ao blob (arquivo) no bucket
    blob = bucket.blob(nome_do_arquivo)

    # Baixe o arquivo para o diretório atual
    blob.download_to_filename(nome_do_arquivo)


In [32]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark pandas numpy

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.2.2-bin-hadoop3.2')


from pyspark.sql import functions as func
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import warnings, re


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


# Instanciando Spark
spark = SparkSession.builder.master('local[*]').getOrCreate()

--2023-04-20 00:48:11--  https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301112604 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop3.2.tgz’

spark-3.2.2-bin-had 100%[===================>] 287.16M  17.0MB/s    in 16s     

2023-04-20 00:48:27 (17.8 MB/s) - ‘spark-3.2.2-bin-hadoop3.2.tgz’ saved [301112604/301112604]



In [42]:
path = "Person.Person.csv"
path1 = "Production.Product.csv"
path2 = "Sales.Customer.csv"
path3 = "Sales.SalesOrderDetail.csv"
path4 = "Sales.SalesOrderHeader.csv"
path5 = "Sales.SpecialOfferProduct.csv"

In [39]:
df_person = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=';').load(path)

In [43]:
df_production = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=';').load(path1)

In [44]:
df_custumer = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=';').load(path2)

In [45]:
df_detail = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=';').load(path3)

In [46]:
df_header = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=';').load(path4)

In [47]:
df_offer_product = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=';').load(path5)

In [40]:
df_person.show()

+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+--------------------+--------------------+--------------------+
|BusinessEntityID|PersonType|NameStyle|Title|FirstName|MiddleName|  LastName|Suffix|EmailPromotion|AdditionalContactInfo|        Demographics|             rowguid|        ModifiedDate|
+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+--------------------+--------------------+--------------------+
|               1|        EM|        0| NULL|      Ken|         J|   Sánchez|  NULL|             0|                 NULL|"<IndividualSurve...|92C4279F-1207-48A...|2009-01-07 00:00:...|
|               2|        EM|        0| NULL|    Terri|       Lee|     Duffy|  NULL|             1|                 NULL|"<IndividualSurve...|D8763459-8AA8-47C...|2008-01-24 00:00:...|
|               3|        EM|        0| NULL|  Roberto|      NULL|Tamburell

In [41]:
df_person.count()

19972

In [48]:
df_production.show(10)

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+--------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|       SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------

In [49]:
df_custumer.show(10)

+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|CustomerID|PersonID|StoreID|TerritoryID|AccountNumber|             rowguid|        ModifiedDate|
+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|         1|    NULL|    934|          1|   AW00000001|3F5AE95E-B87D-4AE...|2014-09-12 11:15:...|
|         2|    NULL|   1028|          1|   AW00000002|E552F657-A9AF-4A7...|2014-09-12 11:15:...|
|         3|    NULL|    642|          4|   AW00000003|130774B1-DB21-4EF...|2014-09-12 11:15:...|
|         4|    NULL|    932|          4|   AW00000004|FF862851-1DAA-404...|2014-09-12 11:15:...|
|         5|    NULL|   1026|          4|   AW00000005|83905BDC-6F5E-4F7...|2014-09-12 11:15:...|
|         6|    NULL|    644|          4|   AW00000006|1A92DF88-BFA2-467...|2014-09-12 11:15:...|
|         7|    NULL|    930|          1|   AW00000007|03E9273E-B193-448...|2014-09-12 11:15:...|
|         8|    NULL

In [50]:
df_detail.show(10)

+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+--------------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|        ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+--------------------+
|       43659|                 1|         4911-403C-98|       1|      776|             1| 2024,994|             0,00| 2024.994|B207C96D-D9E6-402...|2011-05-31 00:00:...|
|       43659|                 2|         4911-403C-98|       3|      777|             1| 2024,994|             0,00| 6074.982|7ABB600D-1E77-41B...|2011-05-31 00:00:...|
|       43659|                 3|         4911-403C-98|       1|      778|             1| 2024,994|             0,00| 2024.994|475CF8C6-49F6-486...|20

In [51]:
df_header.show(10)

+------------+--------------+--------------------+--------------------+--------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+---------------+---------------+------------+------------+----------------------+--------------+----------+---------+---------+----------+-------+--------------------+--------------------+
|SalesOrderID|RevisionNumber|           OrderDate|             DueDate|            ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAddressID|ShipToAddressID|ShipMethodID|CreditCardID|CreditCardApprovalCode|CurrencyRateID|  SubTotal|   TaxAmt|  Freight|  TotalDue|Comment|             rowguid|        ModifiedDate|
+------------+--------------+--------------------+--------------------+--------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+----------

In [52]:
df_offer_product.show(10)

+--------------+---------+--------------------+--------------------+
|SpecialOfferID|ProductID|             rowguid|        ModifiedDate|
+--------------+---------+--------------------+--------------------+
|             1|      680|BB30B868-D86C-455...|2011-04-01 00:00:...|
|             1|      706|B3C9A4B1-2AE6-4CB...|2011-04-01 00:00:...|
|             1|      707|27B711FE-0B77-4EA...|2011-04-01 00:00:...|
|             1|      708|46CBB78B-246E-4D6...|2011-04-01 00:00:...|
|             1|      709|CF102AA0-055F-4D2...|2011-04-01 00:00:...|
|             1|      710|63718DA1-464B-432...|2011-04-01 00:00:...|
|             1|      711|457EB971-D1C9-48C...|2011-04-01 00:00:...|
|             1|      712|5B948448-BAE5-4F2...|2011-04-01 00:00:...|
|             1|      713|07768F40-6E46-430...|2011-04-01 00:00:...|
|             1|      714|85004BCE-C74A-4D4...|2011-04-01 00:00:...|
+--------------+---------+--------------------+--------------------+
only showing top 10 rows



In [56]:
from pyspark.sql.functions import count, col

# calcular a quantidade de linhas por SalesOrderID
df_qnt_linhas_sales = (df_detail
      .groupBy("SalesOrderID")
      .agg(count("*").alias("RowCount")))

# selecionar apenas as linhas que possuem pelo menos três linhas de detalhes
df_qnt_linhas_sales = df_qnt_linhas_sales.where(col("RowCount") >= 3)

# exibir o resultado
df_qnt_linhas_sales.show()


+------------+--------+
|SalesOrderID|RowCount|
+------------+--------+
|       43688|      11|
|       45307|       7|
|       45341|       9|
|       46943|      14|
|       46994|      35|
|       48398|       3|
|       49855|      46|
|       50223|       6|
|       51123|      53|
|       51415|       4|
|       51595|       3|
|       53565|      47|
|       53634|       3|
|       53963|       3|
|       54190|       4|
|       55265|      13|
|       55283|      38|
|       56110|       4|
|       56640|       3|
|       56680|       4|
+------------+--------+
only showing top 20 rows



In [57]:
from pyspark.sql.functions import sum
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# unir as tabelas SalesOrderDetail, SpecialOfferProduct e Product
df_exer_dois = (df_detail
      .join(df_offer_product, "SpecialOfferID")
      .join(df_production, "ProductID"))

# agrupar pelo número de dias para manufatura e pelo nome do produto
# calcular a soma de OrderQty para cada grupo
# criar uma janela ordenada pela soma de OrderQty em ordem decrescente
# e usar a função rank para numerar cada linha na janela
w = Window.partitionBy("DaysToManufacture", "Name").orderBy(F.desc("TotalOrderQty"))
df_exer_dois = (df_exer_dois
      .groupBy("DaysToManufacture", "Name")
      .agg(sum("OrderQty").alias("TotalOrderQty"))
      .withColumn("rank", F.rank().over(w)))

# selecionar as três primeiras linhas para cada grupo
df_exer_dois = df_exer_dois.where(F.col("rank") <= 3)

# exibir o resultado
df_exer_dois.show()


+-----------------+--------------------+-------------+----+
|DaysToManufacture|                Name|TotalOrderQty|rank|
+-----------------+--------------------+-------------+----+
|                0|        AWC Logo Cap|      2109633|   1|
|                0|All-Purpose Bike ...|        73455|   1|
|                0|Bike Wash - Disso...|       820331|   1|
|                0|          Cable Lock|       305577|   1|
|                0|     Classic Vest, L|        61065|   1|
|                0|     Classic Vest, M|       552332|   1|
|                0|     Classic Vest, S|       758309|   1|
|                0|Fender Set - Moun...|       625695|   1|
|                0|Full-Finger Glove...|       517534|   1|
|                0|Full-Finger Glove...|       459194|   1|
|                0|Full-Finger Glove...|       147500|   1|
|                0|    HL Mountain Tire|       411820|   1|
|                0|        HL Road Tire|       253110|   1|
|                0|Half-Finger Glove...|

In [61]:
from pyspark.sql.functions import sum, col
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# fazer join entre as tabelas SalesOrderDetail, SpecialOfferProduct e Product
df_exer_tres = (df_detail
      .join(df_offer_product, "ProductID")
      .join(df_production, "ProductID"))

# calcular a soma de OrderQty para cada produto e número de dias para manufatura
df_exer_tres = (df_exer_tres
      .groupBy("DaysToManufacture", "Name")
      .agg(sum("OrderQty").alias("TotalQty")))

# criar janela ordenada pela soma de OrderQty
w = (Window
     .orderBy(col("TotalQty").desc())
     .partitionBy("DaysToManufacture"))

# selecionar os 3 produtos mais vendidos por cada número de dias para manufatura

df_exer_tres = (df_exer_tres
      .withColumn("Rank", row_number().over(w))
      .where(col("Rank") <= 3)
      .drop("Rank"))


# exibir o resultado
df_exer_tres.show()


+-----------------+--------------------+--------+
|DaysToManufacture|                Name|TotalQty|
+-----------------+--------------------+--------+
|                0|Sport-100 Helmet,...|   33715|
|                0|        AWC Logo Cap|   33244|
|                0|Sport-100 Helmet,...|   32660|
|                1|LL Road Frame - B...|    4743|
|                1|ML Road Frame-W -...|    4305|
|                1|LL Road Frame - R...|    2973|
|                2|HL Mountain Frame...|    2412|
|                2|ML Road Frame-W -...|     895|
|                2|LL Mountain Frame...|     884|
|                4|Mountain-200 Blac...|   11908|
|                4|    Road-650 Red, 44|    9016|
|                4|Mountain-200 Blac...|    7992|
+-----------------+--------------------+--------+



In [63]:
from pyspark.sql.functions import sum

# Join dos dataframes SalesOrderHeader, SalesOrderDetail e Product usando as colunas comuns
sales_df_exe_quatro = df_header.join(df_detail, "SalesOrderID") \
                                .join(df_production, "ProductID")

# Agrupamento por ProductID e OrderDate, com soma de OrderQty
result_df = sales_df_exe_quatro.groupBy("ProductID", "OrderDate") \
                    .agg(sum("OrderQty").alias("TotalOrderQty"))

# Exibição do resultado
result_df.show()


+---------+--------------------+-------------+
|ProductID|           OrderDate|TotalOrderQty|
+---------+--------------------+-------------+
|      767|2011-05-31 00:00:...|            1|
|      749|2011-07-28 00:00:...|            1|
|      749|2011-08-01 00:00:...|           12|
|      751|2011-08-16 00:00:...|            2|
|      715|2011-10-01 00:00:...|          126|
|      768|2011-10-31 00:00:...|           26|
|      772|2011-11-02 00:00:...|            1|
|      749|2011-11-04 00:00:...|            1|
|      775|2011-11-29 00:00:...|            1|
|      764|2011-12-01 00:00:...|           22|
|      752|2011-12-23 00:00:...|            1|
|      774|2012-02-09 00:00:...|            1|
|      753|2012-02-24 00:00:...|            1|
|      777|2012-03-06 00:00:...|            1|
|      749|2012-03-08 00:00:...|            1|
|      749|2012-03-23 00:00:...|            1|
|      751|2012-04-20 00:00:...|            2|
|      752|2012-05-07 00:00:...|            3|
|      783|20

In [69]:
from pyspark.sql.functions import year, month

# carrega a tabela Sales.SalesOrderHeader
sales_orders = df_header

# filtra as linhas que atendem aos critérios especificados
filtered_orders = sales_orders.filter(
    (year("OrderDate") == 2011) & (month("OrderDate") == 9) & (sales_orders["TotalDue"] > 1000)
)

# seleciona as colunas SalesOrderID, OrderDate e TotalDue
selected_columns = filtered_orders.select("SalesOrderID", "OrderDate", "TotalDue")

# ordena pelo TotalDue decrescente
result = selected_columns.orderBy("TotalDue", ascending=False)

# mostra o resultado
result.show()



+------------+---------+--------+
|SalesOrderID|OrderDate|TotalDue|
+------------+---------+--------+
+------------+---------+--------+



In [70]:
# registra a tabela SalesOrderHeader como uma tabela temporária
df_header.createOrReplaceTempView("sales_order_header")

# executa a consulta SQL sobre a tabela temporária
result = spark.sql("""
    SELECT SalesOrderID, OrderDate, TotalDue
    FROM sales_order_header
    WHERE YEAR(OrderDate) = 2011 AND MONTH(OrderDate) = 9 AND TotalDue > 1000
    ORDER BY TotalDue DESC
""")

# mostra o resultado
result.show()



+------------+---------+--------+
|SalesOrderID|OrderDate|TotalDue|
+------------+---------+--------+
+------------+---------+--------+

